In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.rdd.RDD
import org.apache.spark.rdd.PairRDDFunctions._
import org.apache.spark.SparkContext
import org.apache.spark.graphx.Edge
import org.apache.spark.graphx.TripletFields
import org.apache.spark.graphx.Graph
import org.apache.spark.graphx.VertexId
import org.apache.spark.graphx.EdgeTriplet
import org.apache.spark.graphx.GraphLoader
import org.apache.spark._
import org.apache.spark.graphx._
import scala.util.Random
import scala.io.Source
import java.io._

Intitializing Scala interpreter ...

Spark Web UI available at http://3272f0aea206:4040
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1618558554636)
SparkSession available as 'spark'


2021-04-16 07:35:53 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.rdd.RDD
import org.apache.spark.rdd.PairRDDFunctions._
import org.apache.spark.SparkContext
import org.apache.spark.graphx.Edge
import org.apache.spark.graphx.TripletFields
import org.apache.spark.graphx.Graph
import org.apache.spark.graphx.VertexId
import org.apache.spark.graphx.EdgeTriplet
import org.apache.spark.graphx.GraphLoader
import org.apache.spark._
import org.apache.spark.graphx._
import scala.util.Random
import scala.io.Source
import java.io._


In [2]:
// Class to store vertex special attributes
class VertexAttr(
    var degree : Int = 0,
    var initialLabel : Boolean = false,
    var vertexLabels : Array[Float] = Array[Float](0, 0, 0)
) extends java.io.Serializable {     
    
    def setDegree(degree: Int) = {
        this.degree = degree
    }
}

defined class VertexAttr


In [3]:
// Special function to read label file and transform it into a Map object
def readLabel(filename: String) = {
    val pairs = 
        for {
            line <- Source.fromFile(filename).getLines()
            split = line.split(" ").map(_.trim)
            name = split.head.toInt
            scores = split.tail(0).toInt
        } yield (name -> scores)
        pairs.toMap
}

readLabel: (filename: String)scala.collection.immutable.Map[Int,Int]


In [4]:
// Load initial Graph
val initialGraph = GraphLoader.edgeListFile(sc, "./data/email-eu-core/email-Eu-core.txt", canonicalOrientation=false)
val initialLabel = readLabel("./data/email-eu-core/email-Eu-core-department-labels.txt")
val n_label = initialLabel.map({ case (k, v) => v }).max + 1

initialGraph: org.apache.spark.graphx.Graph[Int,Int] = org.apache.spark.graphx.impl.GraphImpl@5c988d47
initialLabel: scala.collection.immutable.Map[Int,Int] = Map(645 -> 1, 892 -> 38, 69 -> 15, 809 -> 22, 629 -> 1, 365 -> 14, 138 -> 21, 760 -> 17, 101 -> 16, 479 -> 1, 347 -> 0, 846 -> 1, 909 -> 4, 333 -> 10, 628 -> 7, 249 -> 14, 893 -> 14, 518 -> 35, 962 -> 9, 468 -> 1, 234 -> 19, 941 -> 41, 0 -> 1, 777 -> 9, 555 -> 14, 666 -> 14, 88 -> 20, 481 -> 22, 352 -> 0, 408 -> 21, 977 -> 19, 170 -> 13, 523 -> 4, 582 -> 15, 762 -> 21, 115 -> 13, 683 -> 22, 730 -> 23, 217 -> 37, 276 -> 4, 994 -> 21, 308 -> 1, 741 -> 7, 5 -> 25, 873 -> 15, 449 -> 1, 120 -> 1, 247 -> 14, 379 -> 7, 878 -> 11, 440 -> 4, 655 -> 4, 511 -> 0, 614 -> 24, 269 -> 17, 677 -> 26, 202 -> 4, 597 -> 23, 861 -> 5, 10 -> 9, 385 ->...

In [5]:
// Setup 
val rand = new Random(42)
val probability = 0.5

def random_label(random: Random): Int = {
    random.nextInt(3)
}

// Transform function to apply to all vertices of the graph
def transform(VD: VertexId): VertexAttr = {
    var labelArray = Array.ofDim[Float](n_label)
    var va = new VertexAttr(0, false, labelArray);
    
    if (initialLabel.contains(VD.toInt)) {
        var label = initialLabel(VD.toInt)
        
        if (rand.nextFloat() < probability){
            labelArray(label) = 1
            va.initialLabel = true
            va.vertexLabels = labelArray
        }
    }
    
    va;
}

rand: scala.util.Random = scala.util.Random@3003e69a
probability: Double = 0.5
random_label: (random: scala.util.Random)Int
transform: (VD: org.apache.spark.graphx.VertexId)VertexAttr


In [6]:
// Transform vertices into VertexAttr with a random label assigned
val graph = initialGraph.mapVertices({ case (id, attr) => transform(id)})

graph: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@5ea46e9e


In [7]:
// Populate Graph degree for each vertices and create final graph
val vertex_rdd = graph.degrees.zip(graph.vertices).map({ case (vDeg, vAttr) => vAttr._2.setDegree(vDeg._2) ; vAttr})
val edge_rdd = graph.edges
val graph2 = Graph(vertex_rdd, edge_rdd)

// Edges go both sides ways
val finalGraph = Graph(graph2.vertices, graph2.edges.union(graph2.edges.reverse))

vertex_rdd: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, VertexAttr)] = MapPartitionsRDD[20] at map at <console>:66
edge_rdd: org.apache.spark.graphx.EdgeRDD[Int] = EdgeRDDImpl[14] at RDD at EdgeRDD.scala:41
graph2: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@505f163f
finalGraph: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@5cd4782c


In [8]:
// Vertex Progam
def VertexProgram(id : VertexId, 
                  VD : VertexAttr, 
                  A : Array[Float]) : VertexAttr  = {
    var newVertexAttr = new VertexAttr()
    newVertexAttr.degree = VD.degree
    newVertexAttr.initialLabel = VD.initialLabel
    newVertexAttr.vertexLabels = VD.vertexLabels
    
    // If the node does not have a label
    if(VD.initialLabel == false){
        newVertexAttr.vertexLabels = A.map(_ / VD.degree)
    }
    newVertexAttr
}

// Send Message
def SendMsg(triplet : EdgeTriplet[VertexAttr, Int]): Iterator[(VertexId, Array[Float])] = {
    Iterator((triplet.dstId, triplet.srcAttr.vertexLabels))
}

// Merge Message
def MergeMsg(A : Array[Float],
             B : Array[Float]) : Array[Float] = {
    val mergeMsg = new Array[Float](A.length);
    for ( i <- 0 to (A.length - 1)) {
        mergeMsg(i) = A(i) + B(i);
    }
    mergeMsg;
}

VertexProgram: (id: org.apache.spark.graphx.VertexId, VD: VertexAttr, A: Array[Float])VertexAttr
SendMsg: (triplet: org.apache.spark.graphx.EdgeTriplet[VertexAttr,Int])Iterator[(org.apache.spark.graphx.VertexId, Array[Float])]
MergeMsg: (A: Array[Float], B: Array[Float])Array[Float]


In [9]:
// finalGraph.vertices.foreach(v => println(v._1, v._2.degree, v._2.initialLabel, v._2.vertexLabels.mkString(" ")))

In [10]:
val initialMsg = Array.ofDim[Float](n_label)
val maxIter = 20   // Stop after maxIter iterations

val graphLP = finalGraph.pregel(initialMsg, maxIter, EdgeDirection.Either)(
    (id, VD, newLabelsArray) => VertexProgram(id, VD, newLabelsArray),
    tr => SendMsg(tr),
    (a, b) => MergeMsg(a, b)
)

initialMsg: Array[Float] = Array(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
maxIter: Int = 20
graphLP: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@6f08e25


In [11]:
graphLP.vertices.foreach(v => println(v._1, v._2.initialLabel, v._2.vertexLabels.mkString(",")))

(454,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(451,false,0.0053939624,0.010616169,2.9264018E-4,8.777788E-4,0.015171738,0.005053575,0.0018828481,0.0051149363,0.0016817392,0.005073114,0.0068955244,0.0032655955,5.124044E-4,0.009232806,0.8151244,0.012733507,0.0054401034,0.0027258724,0.0,0.007866451,0.0022350424,0.004177329,0.0015284277,0.0047439625,3.314097E-5,0.0067693545,0.006286061,6.4174086E-4,5.499347E-4,7.1944E-4,4.515737E-6,4.626206E-4,0.011514083,3.5241778E-6,0.0066211657,0.0022677376,0.019521376,0.013307842,0.0027739056,3.6476715E-4,4.3276767E-4,8.1425635E-5)
(893,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(147,false,0.026815576,0.050476737,7.1183254E-4,0.004597504,0.040154796,0.010094301,0.0067085274,0.015230417,0.006297

(386,false,0.17768538,0.087499656,6.542295E-4,0.0025706047,0.04862386,0.011131138,0.00582528,0.31199107,0.0063155154,0.007837117,0.020411316,0.012504094,0.0023495932,0.0126625635,0.040845513,0.017285798,0.004420002,0.00704269,0.0,0.008371677,0.0031171744,0.0073433835,0.0065079904,0.009182355,1.04307066E-4,0.008166987,0.01365288,0.0042594015,0.0013038239,0.0036546597,1.916902E-5,0.0041253245,0.008450293,1.016385E-5,0.018032435,0.013344938,0.076418795,0.024953686,0.0060638,0.0013279003,0.0031934986,7.3150767E-4)
(522,true,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(772,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(324,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

(934,false,0.013796667,0.10843855,0.011741386,0.015248843,0.049514472,0.006947556,0.014180291,0.015621445,0.008000173,0.02250086,0.030449416,0.0077789053,0.00213239,0.0406102,0.05062964,0.27809924,0.009271078,0.009468401,0.0,0.009429117,0.0060170107,0.008324538,0.0064162756,0.0076480787,0.0033357702,0.0050493027,0.028566815,0.0026153952,0.0017755443,0.0033739884,2.107645E-5,0.002455041,0.0045829006,8.486408E-6,0.02025728,0.024846757,0.11153813,0.026887601,0.022955332,0.005795,0.003167124,4.96572E-4)
(318,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(956,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(12,false,0.017826475,0.016601175,9.15308E-4,0.002344208,0.04186884,0.0065170117,0.0026822838,0.010412904,0.0044939574,0.008843272,0.0139

(801,true,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(429,false,0.007466282,0.007965618,2.0000369E-4,0.0017887604,0.016671605,0.0049037687,0.0013110015,0.0074010906,0.0024307333,0.003731372,0.008520226,0.0036094128,0.0012603827,0.00748315,0.014220021,0.009328898,0.07887973,0.6487604,0.0,0.0113131935,0.00467121,0.014735324,0.0058565717,0.0034212389,1.6363937E-4,0.0041525387,0.034080815,0.001895028,0.0018569435,7.587092E-4,2.5467436E-5,6.530681E-4,0.008284507,4.006092E-6,0.0057777683,0.0051863017,0.057995066,0.007076366,0.0041552437,4.869917E-4,0.0013465219,1.680386E-4)
(815,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(131,false,0.008344468,0.014906284,5.139376E-4,0.0035769471,0.047421947,0.0061327126,0.004285417,0.008791305,0.004862417

(363,false,0.047279056,0.032451626,4.8156388E-4,0.0059150457,0.063559316,0.01054099,0.0056130346,0.028478855,0.007964958,0.011149473,0.0644606,0.01496621,0.0027021824,0.051272903,0.09391317,0.049039163,0.032761898,0.009262704,0.0,0.06344864,0.032966763,0.02021467,0.009274528,0.034638938,7.098612E-5,0.006114949,0.030433169,0.027689144,0.0032234848,0.0044449205,3.0254821E-5,0.0025081937,0.019180749,1.715962E-5,0.054138906,0.0345703,0.08346492,0.024566874,0.01182497,8.833543E-4,0.004272295,2.036081E-4)
(929,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(381,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(645,false,0.67914695,0.18142489,2.160824E-4,9.375671E-4,0.012956015,0.0031769143,8.8188174E-4,0.033065546,6.7215017E-4,0.0017818237,0.00

...

(220,false,0.03122195,0.17698205,7.3070574E-4,0.0068269377,0.07255868,0.020631876,0.00532051,0.036431517,0.015416764,0.015062821,0.10923591,0.1094608,0.004562093,0.08018343,0.02758781,0.034162123,0.005443311,0.006419352,0.0,0.009614692,0.005157426,0.008075389,0.0059292745,0.009524117,9.192949E-5,0.009889095,0.019994395,0.0026248666,0.0013120681,0.0034597858,1.3435027E-5,0.0033338487,0.00571878,5.8667138E-5,0.06030571,0.008569137,0.06274632,0.010607495,0.0086462125,0.002786653,0.0022765165,0.001017053)
(62,false,0.027454989,0.033548225,0.0047035404,0.018779289,0.09052843,0.015570081,0.0094765015,0.02020505,0.015764587,0.018031148,0.056452494,0.028875103,0.006991275,0.04273852,0.08971021,0.055224683,0.036312915,0.035131294,0.0,0.0450453,0.017504927,0.02171704,0.01348819,0.030207738,1.3728376E-4,0.008034036,0.016281916,0.017089838,0.0077864802,0.008168316,2.0550178E-5,0.0103582535,0.016276112,2.6248239E-5,0.018927205,0.0131649105,0.10028926,0.020855872,0.019260293,0.001029726,0.00852427,3

(230,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(800,false,0.0130984895,0.018363107,0.0017184202,0.0049039335,0.052899495,0.014013748,0.0060505494,0.025720721,0.0068362723,0.013051225,0.032218814,0.023113232,0.007445092,0.027493613,0.0482478,0.2883434,0.16752335,0.020936053,0.0,0.027645616,0.0047168056,0.016294474,0.005926941,0.007876281,9.512228E-5,0.007847635,0.018863775,0.0051631387,0.001506038,0.001596408,1.47292485E-5,0.00706284,0.004855373,1.4445553E-5,0.019450204,0.019440562,0.052924156,0.013801009,0.00886157,0.0011227226,0.0027438316,1.9197847E-4)
(802,false,0.031808097,0.029948756,5.880854E-4,0.0054263314,0.31504014,0.016654108,0.01481465,0.013713004,0.03242506,0.011978955,0.03912319,0.016083682,0.0016146542,0.037694365,0.07185261,0.029098988,0.012385247,0.011435472,0.0,0.019957198,0.013959924,0.015477543,0.014016621,0.018795792,8.560456E-5,0.01181

(148,false,0.36759287,0.08245613,3.0464213E-4,0.0031268578,0.13777275,0.015579904,0.0038637884,0.078514315,0.0024546178,0.004551592,0.014959411,0.0045494707,0.0010436968,0.013681579,0.021797625,0.0118697155,0.0034224626,0.0045535676,0.0,0.0070173154,0.0034358706,0.0071901362,0.0028391436,0.009822793,2.4159615E-4,0.019341974,0.017171383,0.0028034986,0.0011321796,0.0010512259,8.595935E-6,0.0010736234,0.0036464923,1.7721602E-5,0.07944482,0.004023171,0.051158518,0.010033521,0.003902024,5.7266996E-4,0.0016219495,3.4847646E-4)
(540,false,0.004140838,0.011793853,8.2964415E-4,0.0017515068,0.019695122,0.0036493316,0.001735986,0.0044761673,0.0045866817,0.021343717,0.011894248,0.005616599,0.0015437462,0.008902533,0.021913158,0.4467883,0.0040665576,0.0049418635,0.0,0.005018608,0.002333558,0.005418862,0.0020957452,0.0025892511,5.1541025E-5,0.004437219,0.017012848,0.0016273016,0.0011286061,0.0040026857,6.939201E-6,0.0054184264,0.075338826,1.4230862E-5,0.008347717,0.013756449,0.25713333,0.0063333283,

(880,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(960,false,0.17579867,0.069837615,5.519506E-4,0.003967778,0.14346705,0.044982232,0.019804746,0.23220287,0.002476538,0.006734664,0.016169861,0.007261657,0.00116236,0.014286679,0.03224786,0.022020299,0.0048060133,0.0050619785,0.0,0.0069425837,0.0029093665,0.006908052,0.0036213498,0.025594067,2.225981E-4,0.006959338,0.013356052,0.0065354076,0.001120965,0.002221808,9.536567E-6,0.0011246823,0.013437962,3.4249406E-5,0.011340271,0.0150456065,0.0638358,0.0076620094,0.0045318035,0.0011299684,0.0017214811,8.847387E-4)
(478,false,0.014295357,0.011958159,5.848409E-4,0.0029432345,0.07247211,0.0102648195,0.004689794,0.011335241,0.22033133,0.048802376,0.057296526,0.011789246,0.0019385639,0.019997178,0.02746526,0.08990958,0.007164702,0.00788177,0.0,0.011960847,0.006725469,0.009760072,0.04292674,0.0075724134,2.2339643E-4,0.006

(521,false,0.014210045,0.040678147,0.24837455,0.015314473,0.069355875,0.011288565,0.004901616,0.0657237,0.0062382505,0.022888172,0.0389398,0.011723433,0.0045674862,0.027901243,0.10310465,0.02363464,0.010420992,0.009166755,0.0,0.015115815,0.006371257,0.009511905,0.00821175,0.007790008,8.0563324E-5,0.011349325,0.013858702,0.00499396,0.00260965,0.0025532185,1.1970925E-5,0.0043487786,0.009354492,2.7158816E-5,0.011907446,0.01060906,0.06531205,0.066070355,0.016520234,0.0010968759,0.003093617,7.624541E-4)
(739,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(233,false,0.008472068,0.015394144,5.462309E-4,0.0034518417,0.025372684,0.006037145,0.0023798652,0.008061687,0.0059633898,0.007824054,0.028962146,0.006593538,0.0030302682,0.034854013,0.04114902,0.01659676,0.025967862,0.0087734535,0.0,0.22994177,0.012347958,0.027826242,0.011362486,0.006208648,1.5593003E-4,0.004743448